## Accessing ESA Climate Change Initiative (CCI) Land Cover with the Planetary Computer STAC API

The [ESA Climate Change Initiative (CCI) Land Cover](https://cds.climate.copernicus.eu/cdsapp#!/dataset/satellite-land-cover?tab=overview) dataset provides consistent global annual land cover maps at 300m spatial resolution from 1992 to 2020. The land cover classes are defined using the United Nations Food and Agriculture Organization's (UN FAO) [Land Cover Classification System (LCCS)](https://www.fao.org/land-water/land/land-governance/land-resources-planning-toolbox/category/details/en/c/1036361/). In addition to the land cover maps, four quality flags are produced to document the reliability of the classification and change detection.

The data in this Collection have been converted from the original NetCDF format to a set of tiled [Cloud Optimized GeoTIFFs (COGs](https://www.cogeo.org/).

Documentation for this dataset is available at the [Planetary Computer Data Catalog](https://planetarycomputer.microsoft.com/dataset/usgs-lcmap-conus-v13).

### Data Access
This notebook works with or without an API key, but you will be given more permissive access to the data with an API key. The [Planetary Computer Hub](https://planetarycomputer.microsoft.com/compute) sets the environment variable "PC_SDK_SUBSCRIPTION_KEY" when your server is started. The API key may be manually set via the following code:

```python
pc.settings.set_subscription_key(<YOUR API Key>)
```

The datasets hosted by the Planetary Computer are available in [Azure Blob Storage](https://docs.microsoft.com/en-us/azure/storage/blobs/). We'll use [pystac-client](https://pystac-client.readthedocs.io/) to search the Planetary Computer's [STAC API](https://planetarycomputer.microsoft.com/api/stac/v1/docs) for the subset of the data that we care about, and then we'll load the data directly from Azure Blob Storage. We'll specify a `modifier` so that we can access the data stored in the Planetary Computer's private Blob Storage Containers. See [Reading from the STAC API](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/) and [Using tokens for data access](https://planetarycomputer.microsoft.com/docs/concepts/sas/) for more.

In [ ]:
import planetary_computer
import pystac_client

# Open the Planetary Computer STAC API
catalog = pystac_client.Client.open(
    "https://pct-apis-staging.westeurope.cloudapp.azure.com/stac/",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("esa-cci-lc")
collection

In [ ]:
# Search the catalog and collection for desired items
latitude = 39.50
longitude = -98.35

Location = [longitude, latitude]
geometry = {
    "type": "Point",
    "coordinates": Location,
}

search = catalog.search(collections=collection, intersects=geometry, datetime="2017")
items = list(search.get_items())
items

### Available Assets & Metadata

Let's display the available [assets](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md#asset-object) and metadata. 

In [ ]:
import rich.table

# Assets
t_assets = rich.table.Table("Key", "Value")
for key, asset in items[0].assets.items():
    t_assets.add_row(key, asset.title)
t_assets

In [ ]:
# Metadata
t_metadata = rich.table.Table("Key", "Value")
for k, v in sorted(items[0].properties.items()):
    t_metadata.add_row(k, str(v))
t_metadata

### Loading the gridded data
Now let's load STAC items into an xarray dataset using [odc-stac](https://github.com/opendatacube/odc-stac).

In [ ]:
import odc.stac

latitude = 39.50
longitude = -98.35
buffer = 5
bbox = [longitude - buffer, latitude - buffer, longitude + buffer, latitude + buffer]


ds = odc.stac.load(items, bbox=bbox)
ds

### Displaying the data

This dataset includes a preferred colormap mapping raster values to colors. The Collection's `item_assets` field includes an overview of the class descriptions and values.

In [ ]:
from pystac.extensions.item_assets import ItemAssetsExtension

ia = ItemAssetsExtension.ext(collection)
x = ia.item_assets["lccs_class"]

class_names = {
    x["description"]: x["value"] for x in x.properties["classification:classes"]
}
class_values = {v: k for k, v in class_names.items()}

t = rich.table.Table("Description", "Value")
for k, v in class_names.items():
    t.add_row(k, str(v))
t

And the Planetary Computer's [Data API](https://planetarycomputer.microsoft.com/api/data/v1/docs) includes the colormap.

In [ ]:
import requests

classmap = requests.get(
    "https://pct-apis-staging.westeurope.cloudapp.azure.com/data/legend/classmap/esa-cci-lc"
).json()
classmap

We'll convert those values to a [matplotlib Colormap](https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.ListedColormap.html) for plotting.

In [ ]:
import matplotlib.colors
import numpy as np

colors = [matplotlib.colors.to_rgba([x / 255 for x in c]) for c in classmap.values()]  #
cmap = matplotlib.colors.ListedColormap(colors, name="esa-cci-lc")
ticks = np.arange(cmap.N)
labels = [class_values.get(value, "nodata") for value in ticks]

Finally, we can read and plot the data. 

In [ ]:
import rioxarray

item = items[0]
ds = rioxarray.open_rasterio(item.assets["lccs_class"].href).squeeze()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 12))

p = ds.plot(
    ax=ax,
    cmap=cmap,
)

ax.set_axis_off()
ax.set_title("ESA CCI \n Baja California Peninsula")

colorbar = fig.axes[1]